In [1]:
import sys
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
import gc

%load_ext autoreload
%autoreload 2

In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [3]:
g = S(4)

In [5]:
s = g.get_subgroups()

In [6]:
s

{frozenset({Permutation(3)}),
 frozenset({Permutation(0, 1)(2, 3),
            Permutation(2, 3),
            Permutation(3),
            Permutation(3)(0, 1)}),
 frozenset({Permutation(0, 1)(2, 3),
            Permutation(0, 2, 1, 3),
            Permutation(0, 2)(1, 3),
            Permutation(0, 3, 1, 2),
            Permutation(0, 3)(1, 2),
            Permutation(2, 3),
            Permutation(3),
            Permutation(3)(0, 1)}),
 frozenset({Permutation(0, 1, 2, 3),
            Permutation(0, 2)(1, 3),
            Permutation(0, 3, 2, 1),
            Permutation(3)}),
 frozenset({Permutation(0, 1, 2, 3),
            Permutation(0, 1)(2, 3),
            Permutation(0, 2)(1, 3),
            Permutation(0, 3, 2, 1),
            Permutation(0, 3)(1, 2),
            Permutation(1, 3),
            Permutation(3),
            Permutation(3)(0, 2)}),
 frozenset({Permutation(0, 3), Permutation(3)}),
 frozenset({Permutation(3), Permutation(3)(0, 1, 2), Permutation(3)(0, 2, 1)}),
 frozens

In [3]:
path = '/workspace/models/2024_08_05_18_31_53_S_5__times_A_5__Z_2__'
models, params = load_models(path, sel='final')

100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


In [4]:
data = GroupData(params)

Intersection size: 7920/14400 (0.55)
Added 7920 elements from intersection
Added 0 elements from group 0: S(5)
Added 0 elements from group 1: times(A(5),Z(2))
Train set size: 7920/14400 (0.55)


In [5]:
def part_var(x, part, dim):
    '''
    Computes E[var(x | part)] / var(x), where part is a partition of x along dim.
    By the law of total variance, this quantity is in [0, 1].
    Stander et al instead compute (\sum_part var(x | part)) / var(x), which is less principled.
    '''
    assert sorted(sum(part, [])) == list(range(x.shape[dim])), "part must be a partition of x along dim"
    numer = sum(
        x.index_select(dim, t.Tensor(p).int().to(x.device)).var(dim=dim, correction=0) * (len(p) / x.shape[dim])
        for p in part
    )
    return numer / x.var(dim=dim, correction=0)

In [6]:
subgroups = {
    i: data.groups[i].get_subgroups_idx()
    for i in [0, 1]
}

Computing subgroups
Computing orders
Computing elements


Computing subgroups: 100%|██████████| 17/17 [02:09<00:00,  7.63s/it]


Computing conjugates
Computing subgroups
Computing orders
Computing elements


Computing subgroups: 100%|██████████| 20/20 [02:14<00:00,  6.70s/it]


Computing conjugates


In [9]:
hash((1, 2, 3, 4, 5))

-5659871693760987716

In [12]:
t.eye(5).tolist()

[[1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0]]

In [13]:
import hashlib

In [14]:
m = hashlib.sha256()

In [16]:
m.update('test test test'.encode())

In [17]:
m.hexdigest()

'497b22d4e86a3caa9f5baa24435a99ac1154094a0b9302b9bcd9d6544d6efbe9'

In [19]:
str(t.eye(5).int().tolist())

'[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]'